In [ ]:
# 🚀 STEP 1 — Install Required Libraries
!pip install pathway bokeh panel geopy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 🚀 STEP 2 — Imports
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
from geopy.distance import geodesic
from google.colab import files

In [ ]:
# 🚀 STEP 3 — Upload CSV
uploaded = files.upload()  # Upload 'dataset.csv'

Saving dataset.csv to dataset.csv


In [ ]:
# 🚀 STEP 4 — Load & Clean CSV
df = pd.read_csv("dataset.csv")
# Add fallback and transformation logic
if "TrafficConditionNearby" in df.columns:
    df["Traffic"] = df["TrafficConditionNearby"]

traffic_map = {"low": 1.0, "average": 2.0, "high": 3.0}
df["Traffic"] = df["Traffic"].map(traffic_map).fillna(1.0)

df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df["TimestampSeconds"] = df["Timestamp"].astype("int64") // 10**9

# Ensure all columns exist
for col, default in [("QueueLength", 0), ("IsSpecialDay", 0), ("VehicleType", "car")]:
    df[col] = df.get(col, default)

df["Occupancy"] = pd.to_numeric(df["Occupancy"], errors="coerce").fillna(0).astype(int)
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce").fillna(1).astype(int)
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce").fillna(0).astype(int)
df["Traffic"] = pd.to_numeric(df["Traffic"], errors="coerce").fillna(1.0).astype(float)
df["IsSpecialDay"] = pd.to_numeric(df["IsSpecialDay"], errors="coerce").fillna(0).astype(int)
df["TimestampSeconds"] = pd.to_numeric(df["TimestampSeconds"], errors="coerce").fillna(0).astype(int)
df["VehicleType"] = df["VehicleType"].astype(str)

# Add Lat, Long placeholders if not present
df["Latitude"] = df.get("Latitude", 26.1 + 0.01 * (df.index % 14))
df["Longitude"] = df.get("Longitude", 91.7 + 0.01 * (df.index % 14))
df["LotID"] = df.get("LotID", df.index % 14)

# Save final CSV
df[[
    "LotID", "TimestampSeconds", "Occupancy", "Capacity", "QueueLength",
    "Traffic", "IsSpecialDay", "VehicleType", "Latitude", "Longitude"
]].to_csv("parking_stream_model3.csv", index=False)

In [ ]:
# 🚀 STEP 5 — Define Schema
class ParkingSchema3(pw.Schema):
    LotID: int
    TimestampSeconds: int
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: float
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float


In [ ]:
data = pw.demo.replay_csv(
    "parking_stream_model3.csv",
    schema=ParkingSchema3,
    input_rate=1000
)


In [ ]:
# 🚀 STEP 7 — Feature Engineering
data = data.with_columns(
    occ = pw.this.Occupancy,
    cap = pw.this.Capacity,
    queue = pw.this.QueueLength,
    traffic = pw.this.Traffic,
    special = pw.this.IsSpecialDay,
    vtype = pw.this.VehicleType,
    lot_id = pw.this.LotID,
    lat = pw.this.Latitude,
    lon = pw.this.Longitude,
    timestamp_parsed = pw.this.TimestampSeconds
)

In [ ]:
# 🚀 STEP 8 — Define Demand Function
@pw.udf
def vehicle_weight(vtype: str) -> float:
    weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}
    return weights.get(vtype, 1.0)

ALPHA, BETA, GAMMA, DELTA, EPSILON = 1.0, 0.5, 0.3, 0.2, 0.4

data = data.with_columns(
    demand_raw = (
        ALPHA * (pw.this.occ / pw.this.cap)
        + BETA * pw.this.queue
        - GAMMA * pw.this.traffic
        + DELTA * pw.this.special
        + EPSILON * vehicle_weight(pw.this.vtype)
    )
)


In [ ]:
# 🚀 STEP 9 — Aggregate and Normalize Demand
window = pw.temporal.tumbling(1800)  # 30-minute windows

agg = (
    data.windowby(pw.this.timestamp_parsed, window=window)
    .reduce(
        t = pw.reducers.max(pw.this.timestamp_parsed),
        dmin = pw.reducers.min(pw.this.demand_raw),
        dmax = pw.reducers.max(pw.this.demand_raw),
        dsum = pw.reducers.sum(pw.this.demand_raw),
        dcount = pw.reducers.count(),
        lot_id = pw.reducers.any(pw.this.lot_id),
        lat = pw.reducers.any(pw.this.lat),
        lon = pw.reducers.any(pw.this.lon)
    )
)

agg = agg.with_columns(
    demand_norm = pw.apply(
        lambda avg, dmin, dmax: 0.0 if dmax - dmin == 0 else (avg - dmin) / (dmax - dmin),
        agg.dsum / agg.dcount,
        agg.dmin,
        agg.dmax
    )
)


In [ ]:
# 🚀 STEP 10 — FINAL CLEAN VERSION ✅

# Step 0: UDFs
@pw.udf
def geo_distance(lat1, lon1, lat2, lon2) -> float:
    from geopy.distance import geodesic
    return geodesic((lat1, lon1), (lat2, lon2)).meters

@pw.udf
def to_float(x) -> float:
    return float(x)

@pw.udf
def adjust_price(my_price, avg_price):
    if avg_price is None:
        return my_price
    if my_price > avg_price:
        return my_price - 0.5
    elif my_price < avg_price:
        return my_price + 0.5
    return my_price

# Step 1: Self-join with copy
agg_left = agg
agg_right = agg.copy()

competitor_pairs = agg_left.join(agg_right).select(
    lot_id_left = pw.left.lot_id,
    lot_id_right = pw.right.lot_id,
    lat_left = pw.left.lat,
    lon_left = pw.left.lon,
    lat_right = pw.right.lat,
    lon_right = pw.right.lon,
    price_right = pw.right.price,
    t_left = pw.left.t,
    t_right = pw.right.t
).filter(
    (pw.this.lot_id_left != pw.this.lot_id_right) &
    (pw.this.t_left == pw.this.t_right)
)

# Step 2: Compute distance + cast price
competitor_pairs = competitor_pairs.with_columns(
    distance = geo_distance(
        pw.this.lat_left, pw.this.lon_left,
        pw.this.lat_right, pw.this.lon_right
    ),
    price_right_float = to_float(pw.this.price_right)
).filter(pw.this.distance < 300)

# Step 3: Aggregate competitor prices
avg_comp_price = (
    competitor_pairs.groupby(pw.this.lot_id_left, pw.this.t_left)
    .reduce(
        sum_price = pw.reducers.sum(pw.this.price_right_float),
        count = pw.reducers.count(),
        lot_id = pw.reducers.any(pw.this.lot_id_left),
        t = pw.reducers.any(pw.this.t_left)
    )
).with_columns(
    avg_price = pw.this.sum_price / pw.this.count
)

# Step 4: Join competitor prices to agg
joined = agg.join(avg_comp_price).select(
    **{col: pw.left[col] for col in agg.schema.keys()},
    avg_price = pw.right.avg_price,
    lot_id_right = pw.right.lot_id,
    t_right = pw.right.t
).filter(
    (pw.this.lot_id == pw.this.lot_id_right) & (pw.this.t == pw.this.t_right)
)

# Step 5A: Compute intermediate price based on competitor comparison
agg = joined.with_columns(
    price_competitive = adjust_price(pw.this.price, pw.this.avg_price)
)

# Step 5B: Apply bounds (min = 0.5x, max = 2x base)
agg = agg.with_columns(
    final_price = pw.apply(
        lambda p: min(max(p, BASE_PRICE * 0.5), BASE_PRICE * 2),
        pw.this.price_competitive
    )
)



In [ ]:


# 🚀 STEP 11 — Visualization
pn.extension()

def model3_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800,
        title="Model 3: Competitive Dynamic Pricing",
        x_axis_label="Time (s)", y_axis_label="Final Price ($)",
        x_axis_type="linear"
    )
    fig.line("t", "final_price", source=source, line_width=2, color="green")
    fig.scatter("t", "final_price", source=source, size=6, color="orange")
    return fig

viz = agg.plot(model3_plotter, sorting_col="t")

In [ ]:
# 🚀 STEP 12 — Run Pipeline
%%capture --no-display
pw.run()

Output()